<a href="https://colab.research.google.com/github/MuhammadHassan667410/160-Days-of-learning-complete-AI/blob/main/Phase%202%20%3A%20Machine%20Learning%2C(Day%2020%20to%2050)/Recommenders%20System(Day%2039%20to%2042)/Hybrid_Recommenders_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install lightfm streamlit

In [2]:
import pandas as pd

ratings = pd.read_csv('/content/sample_data/ratings.csv')
movies = pd.read_csv('/content/sample_data/movies.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:

from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(users=ratings['userId'], items=ratings['movieId'])

In [5]:

interactions, weights = dataset.build_interactions(ratings.iloc[:, 0:3].values)

In [6]:

from lightfm import LightFM

model = LightFM(loss='warp')
model.fit(interactions, epochs=10)

In [8]:
import numpy as np

def recommend_movies(user_id, model, interactions, movies, num_recommendations=5):
    n_users, n_items = interactions.shape
    user_id_map = dataset.mapping()[0]

    if user_id not in user_id_map:
        print(f"User ID {user_id} is not in the dataset.")
        return pd.DataFrame()

    user_idx = user_id_map[user_id]
    item_indices = np.arange(n_items, dtype=np.int32)
    scores = model.predict(user_idx, item_indices)
    top_indices = scores.argsort()[-num_recommendations:][::-1]
    item_id_map = {item_idx: item_id for item_id, item_idx in dataset.mapping()[2].items()}

    recommended_movie_ids = [item_id_map[idx] for idx in top_indices]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]

    return recommended_movies[['title']]
enter = input("Enter the user id: ")
enter = int(enter)
recommended_movies_df = recommend_movies(enter, model, interactions, movies)
display(recommended_movies_df)

Enter the user id: 77


,title
224,Star Wars: Episode IV - A New Hope (1977)
257,Pulp Fiction (1994)
277,"Shawshank Redemption, The (1994)"
314,Forrest Gump (1994)
1939,"Matrix, The (1999)"
